# Imports

In [1]:
from typing import List

import re

import pandas as pd
pd.set_option('max_colwidth', 150)

from stop_words import get_stop_words
from nltk import ngrams
from bulstem.stem import BulStemmer 
from lemmagen3 import Lemmatizer 

In [2]:
DATA_PATH_PREP = '../DATA/prepared'

STEM_RULES = './stem_rules_context_2_utf8.txt'

lemmatizer = Lemmatizer('bg')
stemmer = BulStemmer.from_file(STEM_RULES, min_freq=2, left_context=2)

# Load data

In [3]:
df_full = pd.read_pickle(f'{DATA_PATH_PREP}/02_df_full_text_no_meta.pkl')
df_full.head()

,author,title,download_link,local_filename,text
0,ivan_vazov,Чичовци,https://chitanka.info/text/3757-chichovtsi.txt.zip,../DATA/original/ivan_vazov_tbc_chichovtsi.txt,\n\tГалерия от типове и нрави български в турско време\n\n\n\tI. Общество\n\n\tУтринното лятно слънце изскокна високо над Стара планина. Потоци от...
1,ivan_vazov,Под игото,https://chitanka.info/text/3753-pod-igoto.txt.zip,../DATA/original/ivan_vazov_tbc_pod_igoto.txt,"\n\n\n\tЧаст първа\n\n\tI. Гост\n\n\tТая прохладна майска вечер чорбаджи Марко, гологлав, по халат, вечеряше с челядта си на двора.\n\tГосподарска..."
2,ivan_vazov,Българският език,https://chitanka.info/text/5189-bylgarskijat-ezik.txt.zip,../DATA/original/ivan_vazov_bylgarskijat_ezik.txt,"\n\n\n\n\tЕзик свещен на моите деди\n\tезик на мъки, стонове вековни,\n\tезик на тая, дето ни роди\n\tза радост не — за ядове отровни.\n\n\tЕзик п..."
3,ivan_vazov,Кочо,https://chitanka.info/text/3851-kocho.txt.zip,../DATA/original/ivan_vazov_kocho.txt,"\n\tЗащитата на Перущица\n\n\n\tO, движенье славно, о, мрачно движенье,\n\tдни на борба горда, о, дни на паденье!\n\tЕпопея тъмна, непозната нам,\..."
4,ivan_vazov,Левски,https://chitanka.info/text/3849-levski.txt.zip,../DATA/original/ivan_vazov_levski.txt,"\n\n\n\tМанастирът тесен за мойта душа е.\n\tКога човек дойде тук да се покае,\n\tтрябва да забрави греховния мир,\n\tда бяга съблазни и да търси ..."


# Tokenize

In [4]:
def tokenize(raw_text: str, level: str) -> List[str]:
    if level not in {'soft', 'medium', 'hard'}:
        raise NotImplemented(f'Level {level} is not supported')
    
    text = raw_text.lower()
    tokens = re.findall(r"[\w']+|[^\w\s]", text)

    if level in {'medium', 'hard'}:
        # Now: ['песни', 'македония', 'българският', 'бог', ..
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

    if level in {'hard'}:
        # Now: ['песен', 'македония', 'български', 'бог', ..
        tokens = [stemmer.stem(token) for token in tokens]
        # Now: ['песен', 'македони', 'българск', 'бог', ..

    bi_tri_grams = list(ngrams(tokens, 2)) + list(ngrams(tokens, 3)) + list(ngrams(tokens, 4))
    tokens += map(lambda bts: ' '.join(bts), bi_tri_grams)

    return tokens

In [5]:
test = 'Това е низ! Съдържа препинателни "знаци" и цифри като: 42, 420, 4200 и 7.\nКак можем да го разделим??'

print('################ Soft ################')
print(tokenize(test, 'soft'))
print()

print('################ Medium ################')
print(tokenize(test, 'medium'))
print()

print('################ Hard ################')
print(tokenize(test, 'hard'))

################ Soft ################
['това', 'е', 'низ', '!', 'съдържа', 'препинателни', '"', 'знаци', '"', 'и', 'цифри', 'като', ':', '42', ',', '420', ',', '4200', 'и', '7', '.', 'как', 'можем', 'да', 'го', 'разделим', '?', '?', 'това е', 'е низ', 'низ !', '! съдържа', 'съдържа препинателни', 'препинателни "', '" знаци', 'знаци "', '" и', 'и цифри', 'цифри като', 'като :', ': 42', '42 ,', ', 420', '420 ,', ', 4200', '4200 и', 'и 7', '7 .', '. как', 'как можем', 'можем да', 'да го', 'го разделим', 'разделим ?', '? ?', 'това е низ', 'е низ !', 'низ ! съдържа', '! съдържа препинателни', 'съдържа препинателни "', 'препинателни " знаци', '" знаци "', 'знаци " и', '" и цифри', 'и цифри като', 'цифри като :', 'като : 42', ': 42 ,', '42 , 420', ', 420 ,', '420 , 4200', ', 4200 и', '4200 и 7', 'и 7 .', '7 . как', '. как можем', 'как можем да', 'можем да го', 'да го разделим', 'го разделим ?', 'разделим ? ?', 'това е низ !', 'е низ ! съдържа', 'низ ! съдържа препинателни', '! съдържа препин

In [6]:
df_full_no_meta = df_full.drop(['title', 'download_link', 'local_filename'], axis=1)
df_full_no_meta = df_full_no_meta[['text', 'author']]

df_full_soft = df_full_no_meta.copy()
df_full_soft['text'] = df_full_soft['text'].map(lambda t: tokenize(t, 'soft'))

df_full_med = df_full_no_meta.copy()
df_full_med['text'] = df_full_med['text'].map(lambda t: tokenize(t, 'medium'))

df_full_hard = df_full_no_meta.copy()
df_full_hard['text'] = df_full_hard['text'].map(lambda t: tokenize(t, 'hard'))

In [7]:
print('################ Heads ################')
print(df_full_soft['text'][0][:20])
print(df_full_med['text'][0][:20])
print(df_full_hard['text'][0][:20])

print('################ Tails ################')
print(df_full_soft['text'][0][-20:])
print(df_full_med['text'][0][-20:])
print(df_full_hard['text'][0][-20:])

################ Heads ################
['галерия', 'от', 'типове', 'и', 'нрави', 'български', 'в', 'турско', 'време', 'i', '.', 'общество', 'утринното', 'лятно', 'слънце', 'изскокна', 'високо', 'над', 'стара', 'планина']
['галерия', 'от', 'тип', 'и', 'нравя', 'български', 'в', 'турско', 'време', 'i', '.', 'общество', 'утринен', 'лятно', 'слънце', 'изскокна', 'високо', 'над', 'старая', 'планина']
['галери', 'от', 'тип', 'и', 'нравя', 'българск', 'в', 'турск', 'време', 'i', '.', 'обществ', 'утрин', 'лятн', 'слънц', 'изскокн', 'висок', 'над', 'стара', 'плани']
################ Tails ################
['потвърди досетливият хаджи смион', 'досетливият хаджи смион ,', 'хаджи смион , като', 'смион , като си', ', като си изу', 'като си изу лявата', 'си изу лявата калевра', 'изу лявата калевра и', 'лявата калевра и погледна', 'калевра и погледна ласкаво', 'и погледна ласкаво онбашия', 'погледна ласкаво онбашия .', 'ласкаво онбашия . 20', 'онбашия . 20 август', '. 20 август ,', '20 август , 1884

In [8]:
df_full_hard.head()

,text,author
0,"[галери, от, тип, и, нравя, българск, в, турск, време, i, ., обществ, утрин, лятн, слънц, изскокн, висок, над, стара, плани, ., пото, от, заря, ми...",ivan_vazov
1,"[част, първ, i, ., гост, тая, прохла, майск, вечер, чорбадж, марк, ,, гологлав, ,, по, хала, ,, вечеря, с, челяд, си, на, двор, ., господарск, тра...",ivan_vazov
2,"[ези, свещен, на, мои, деди, ези, на, мъки, ,, стон, веков, ,, ези, на, тая, ,, дето, наш, родя, за, радост, не, —, за, яд, отров, ., ези, прекрас...",ivan_vazov
3,"[защит, на, перущиц, o, ,, движенье, славно, ,, о, ,, мрачно, движенье, ,, ден, на, борб, горд, ,, о, ,, ден, на, паденье, !, епоп, тъмен, ,, непо...",ivan_vazov
4,"[манастир, тесен, за, мой, душа, е, ., кога, чове, дойд, тук, да, се, пока, ,, трябва, да, забрав, грехов, мир, ,, да, бяга, съблазн, и, да, търс,...",ivan_vazov


# Saving to files

In [9]:
df_full_soft.to_pickle(f'{DATA_PATH_PREP}/03_df_full_soft.pkl')
df_full_med.to_pickle(f'{DATA_PATH_PREP}/03_df_full_med.pkl')
df_full_hard.to_pickle(f'{DATA_PATH_PREP}/03_df_full_hard.pkl')